In [25]:
import json
import numpy as np
import pandas as pd
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP

YEAR="2012"
EMBEDDINGS_PATH = f"data/embeddings_sentences_with_dates_{YEAR}.npz"
CSV_PATH = f"data/sentences_with_dates_{YEAR}.csv"

In [2]:
# Load json file with stopwords as a list
with open("data/stopwords_cs.json", "r") as f:
    stopwords_cs = json.load(f)
    
stopwords_cs.extend(["moderátorka", "moderátor", "redaktorka", "redaktor"])


In [26]:
sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
custom_umap_model = UMAP(n_components=2, metric='cosine', random_state=3859, low_memory=True)
vectorizer_model = CountVectorizer(stop_words=stopwords_cs)
topic_model = BERTopic(embedding_model=sentence_model, umap_model=custom_umap_model, vectorizer_model=vectorizer_model, min_topic_size = 10, calculate_probabilities=False, verbose=True, low_memory=True)

In [27]:
# Load npz file as an array and csv file as a list
embeddings = np.load(EMBEDDINGS_PATH)['arr_0']
sentences = pd.read_csv(CSV_PATH)["sentence"].tolist()
# Create random index of length 1000
random_index = np.random.choice(len(sentences), 1000, replace=False)


# dates = pd.read_csv(CSV_PATH)["date"].tolist()
# array_2 = np.concatenate((sentence_model.encode(["Ahoj.",  "Co je."]), sentence_model.encode(["Jak se máte.",  "čau lidi."])), axis=0)

In [28]:
# get all properties of embeddings
sentences

['reforma vysokého školství.',
 'daniel takáč moderátor vysoké školy by měly fungovat podle nových pravidel alespoň na nich začíná pracovat vláda.',
 'slíbila to totiž v programovém prohlášení.',
 'zatím je vláda ve fázi psaní záměrů jak by chtěla vysokoškolské vzdělávání organizovat ale už přespříští školní rok by chtěla aby pravidla platila tedy ještě předtím je musí přepsat do paragrafu a prosadit v parlamentu což není zrovna krátká záležitost.',
 'už v té obecné fázi ale kabinet naráží.',
 'záměry se nelíbí ani řadě pracovníků vysokých škol ani mnohým studentům.',
 'proč se jim nelíbí?',
 'co je na vládních záměrech špatně a co je naopak rozumné a přijatelné?',
 'právě o tom si můžete popovídat v dnešním hyde parku a nejenom o tom s prorektorem univerzity karlovy stanislavem štechem.',
 'dobrý večer.',
 'pane profesore dobrý večer.',
 'vítejte u nás.',
 'stanislav štech prorektor pro rozvoj univerzita karlova dobrý večer.',
 'daniel takáč moderátor díky že jste do hyde parku přišel

In [29]:
topics, probs = topic_model.fit_transform(documents=sentences, embeddings=embeddings)

2022-11-14 15:43:15,745 - BERTopic - Reduced dimensionality
2022-11-14 15:43:16,012 - BERTopic - Clustered reduced embeddings


In [30]:
topic_model.visualize_barchart(n_words=30, width=400, height=500, top_n_topics=20)

In [ ]:
# If probabilities were calculated, then the topics can be visualized with the following code:
# topic_model.visualize_distribution(probs[0])
topic_model.get_representative_docs(2)
topic_model.visualize_heatmap()

In [ ]:
topics_over_time = topic_model.topics_over_time(sentences, dates, nr_bins=20)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time)